In [1]:
import requests
import json 
import plotly.express as px
import pandas as pd
import numpy as np

** Rouen Parking API**

+ Using this API to get parking informaiton : https://data.strasbourg.eu/explore/dataset/parkings/table/
+ Using this API to get real time information https://data.opendatasoft.com/explore/dataset/occupation-parkings-temps-reel%40eurometrostrasbourg/api/: 

In [2]:
Parking_info = requests.get("https://data.strasbourg.eu/api/explore/v2.1/catalog/datasets/parkings/records?limit=100&timezone=europe%2FParis") 
Parking_info

<Response [200]>

In [3]:
Dump_parking_info = json.dumps(Parking_info.json())
Loads_parking_info = json.loads(Dump_parking_info)  
parking_info = pd.json_normalize(Loads_parking_info['results'])
parking_info.head(2)

,city,friendlyurl,districtcode,description,address,types,name,idsurfs,zipcode,citycode,...,street,accessforwheelchair,accessfordeaf,normalizedalias,exceptions,accessforblind,schedulelinkurl,hasurlschedule,position.lon,position.lat
0,Strasbourg,https://www.strasbourg.eu/lieu/-/entity/id/452...,SQ_116,Afin de faciliter le stationnement de votre vé...,4 Rue Catherine Kany 67000 Strasbourg France,Cat_22_02,Parking Rhénapark,2471_DEP_64,67000,C_482,...,4 Rue Catherine Kany,1,0,parking-rhenapark,None,0,{},0,7.792645,48.570974
1,Strasbourg,https://www.strasbourg.eu/lieu/-/entity/id/687...,SQ_107,Parking courte durée Le parking Opéra Broglie...,Impasse de Bischeim 67000 Strasbourg France,Cat_22_01,Parking Centre Opéra Broglie,1378_DEP_4,67000,C_482,...,Impasse de Bischeim,1,0,parking-centre-opera-broglie,None,0,{},0,7.749275,48.585689


**Real Time information**

In [4]:


from datetime import datetime 
import pytz 
UTC = pytz.utc
time_At_P = pytz.timezone('Europe/Paris') 
dt_time_P = datetime.now(time_At_P) 
print(dt_time_P.strftime("%Y-%m-%d %H:%M:%S"))

2024-11-27 17:26:38


In [5]:
Parking_reel_tine = requests.get("https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/occupation-parkings-temps-reel@eurometrostrasbourg/records?limit=100&timezone=europe%2FParis") 
Parking_reel_tine

<Response [200]>

In [6]:
Dump_RT_parking = json.dumps(Parking_reel_tine.json())
Loads_RT_parking = json.loads(Dump_RT_parking)  
RT_parking = pd.json_normalize(Loads_RT_parking['results'])

#Compute data
RT_parking["Occupee"] = RT_parking["total"]  - RT_parking["libre"] 
RT_parking["percentage_occupe"] = RT_parking["Occupee"]  / RT_parking["total"]  

RT_parking.head(3)

,nom_parking,idsurfs,ident,etat,etat_descriptif,total,libre,infousager,realtimestatus,infoappli,taux_occup,couleur_occup,position.lon,position.lat,position,Occupee,percentage_occupe
0,Parking Gare Wodli,1382_DEP_8,10,1,Ouvert,955,5,COMPLET,RED,5,0.994764,noir,7.736771,48.587385,NaN,950,0.994764
1,Parking Zénith,1400_DEP_26,1,2,Fermé,3175,3175,0,BLACK,Fermé,0.000000,vert,7.686467,48.595335,NaN,0,0.000000
2,Parking Centre Opéra Broglie,1378_DEP_4,3,2,Fermé,447,200,0,BLACK,Fermé,0.552573,vert,7.749275,48.585689,NaN,247,0.552573


In [7]:
parking_info = parking_info[["name" , "position.lon",	"position.lat"]]
RT_parking = RT_parking.merge(parking_info, how='inner' , left_on='nom_parking' , right_on = 'name')

In [8]:
RT_parking
#RT_parking.loc[RT_parking['etat_descriptif'] == "Ouvert"]
RT_parking_ouvert = RT_parking.loc[RT_parking['etat_descriptif'] == "Ouvert"]
RT_parking_ferme = RT_parking.loc[RT_parking['etat_descriptif'] == "Fermé"]
RT_parking_not_available = RT_parking.loc[RT_parking['etat_descriptif'] == "frequentation temps reel indisponible"]

# Top 3 sites with bigger fullness

In [9]:
RT_parking_ouvert.sort_values(by='percentage_occupe', ascending=False).head(3)

,nom_parking,idsurfs,ident,etat,etat_descriptif,total,libre,infousager,realtimestatus,infoappli,taux_occup,couleur_occup,position.lon_x,position.lat_x,position,Occupee,percentage_occupe,name,position.lon_y,position.lat_y
0,Parking Gare Wodli,1382_DEP_8,10,1,Ouvert,955,5,COMPLET,RED,5,0.994764,noir,7.736771,48.587385,NaN,950,0.994764,Parking Gare Wodli,7.736771,48.587385
5,Parking Austerlitz,1375_DEP_1,21,1,Ouvert,482,25,20,RED,25,0.948133,rouge,7.751796,48.577964,NaN,457,0.948133,Parking Austerlitz,7.751796,48.577964
16,Parking Rotonde,1394_DEP_20,7,1,Ouvert,450,27,20,RED,27,0.940000,rouge,7.726101,48.591226,NaN,423,0.940000,Parking Rotonde,7.726101,48.591226


## Average % occupation

In [10]:
round(sum(RT_parking["Occupee"]) / sum(RT_parking["total"]) ,3)

0.597

## Numbre place dispo/occupe

In [11]:
NBR_place_libre = sum(RT_parking["libre"])
NBR_place_occupe = sum(RT_parking["Occupee"])
nbr_place_total  = sum(RT_parking["total"])
print(NBR_place_libre,NBR_place_occupe , nbr_place_total)

6408 9481 15889


## Nombre parking close

In [12]:
#close_site = RT_parking_not_available["nom_parking"].unique()

close_site = RT_parking_ferme["nom_parking"].unique()
' ,'.join(close_site)

'Parking Zénith ,Parking Centre Opéra Broglie ,Parking Gutenberg ,Parking Kleber Homme de fer'

## Bar plot occupation


In [16]:
#import package
import plotly.io as pio

#And the following will do the trick:
pio.templates.default = "plotly"

In [17]:
a= RT_parking_ouvert.sort_values('percentage_occupe', ascending=False)['nom_parking'].to_list()

In [18]:
Sorting_order = RT_parking_ouvert.sort_values('percentage_occupe', ascending=False)['nom_parking'].to_list()
customdata = np.dstack((RT_parking_ouvert['total']))

fig1 = make_subplots(shared_yaxes=True, shared_xaxes=True  )
fig1.add_bar(x=RT_parking_ouvert['nom_parking'],y=RT_parking_ouvert['total'],opacity=0.6,width=0.95,name='total',hovertemplate='%{y}',marker_color='grey')
fig1.add_bar(x=RT_parking_ouvert['nom_parking'],y=RT_parking_ouvert['Occupee'],width=0.95,name='Occupee',marker_color=RT_parking_ouvert['realtimestatus'] )
fig1.update_layout(barmode='overlay', 
                  title= "Occupation parking Strasbourg",
                  yaxis_title='Occupation',
                    xaxis={'categoryorder':'array', 'categoryarray':Sorting_order},)

fig1.update_traces(
    hovertemplate = "<b> " + RT_parking_ouvert['nom_parking'] + " </b><br>"   +  
                   "<b>État : </b> " + RT_parking_ouvert['etat_descriptif'] + " </b><br>"   + 
                  "<b>Places : %{y} </b> " )


NameError: name 'make_subplots' is not defined

## Maps creation


In [19]:
RT_parking_ouvert.head(3)

,nom_parking,idsurfs,ident,etat,etat_descriptif,total,libre,infousager,realtimestatus,infoappli,taux_occup,couleur_occup,position.lon_x,position.lat_x,position,Occupee,percentage_occupe,name,position.lon_y,position.lat_y
0,Parking Gare Wodli,1382_DEP_8,10,1,Ouvert,955,5,COMPLET,RED,5,0.994764,noir,7.736771,48.587385,NaN,950,0.994764,Parking Gare Wodli,7.736771,48.587385
3,Parking Baggersee,1376_DEP_2,4,1,Ouvert,460,64,60,ORANGE,64,0.860870,orange,7.735695,48.544647,NaN,396,0.860870,Parking Baggersee,7.735695,48.544647
4,Parking Bateliers,1377_DEP_3,2,1,Ouvert,283,29,20,RED,29,0.897527,orange,7.756048,48.581002,NaN,254,0.897527,Parking Bateliers,7.756048,48.581002


In [22]:
fig2 = px.scatter_mapbox(RT_parking_ouvert, 
                        lat=RT_parking_ouvert['position.lat_y'],
                        lon=RT_parking_ouvert['position.lon_y'], 
                        hover_name="nom_parking", 
                        size="etat" ,
                        #hover_data=["status" , "Bike available" , "Available space" ],
                        custom_data=['nom_parking', 'etat_descriptif', 'libre' , 'Occupee','percentage_occupe'] ,
                        #color="percentage_occupe",
                        color_discrete_sequence=[RT_parking_ouvert.realtimestatus],
                        opacity = 0.8,
                        zoom=11, 
                        height=450)
fig2.update_layout(mapbox_style="carto-positron")
fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig2.update_layout(coloraxis_showscale=False)
fig2.update_traces(
    hovertemplate = 
                "<b>%{customdata[0]}</b><br>"   +  
                "<b>État : </b> %{customdata[1]}<br>" + 
                "<b>Parking libre: </b> %{customdata[2]}<br>" +
                "<b>Parking occupe: </b> %{customdata[3]}<br>"  +
                "<b>Pourcentage occupation: </b> %{customdata[4]}%<br>"  )
fig2.show()

## Status parking

In [23]:
RT_parking
etat_data = RT_parking.groupby(['etat_descriptif'], as_index=False).size()
#DataFrame({'count' : RT_parking.groupby( [ "etat_descriptif"] ).size()}).reset_index()
etat_data["share_size"] = round((etat_data['size'] / etat_data['size'].sum()) ,2)
etat_data

,etat_descriptif,size,share_size
0,Fermé,4,0.15
1,Ouvert,19,0.70
2,frequentation temps reel indisponible,4,0.15


In [25]:
status_bar = px.bar(etat_data, y='etat_descriptif',
            x='size' ,
            width=600,
            height=250,
            orientation='h' ,
            color = 'size' ,
            color_continuous_scale=px.colors.sequential.OrRd[::-1])
status_bar.update_layout(yaxis={'categoryorder':'total ascending'},
                  margin=dict(l=20, r=20, t=20, b=20),) 
status_bar.update_layout(coloraxis_showscale=False)
status_bar.show()

In [27]:
#import sys
#print(sys.version)

3.11.2 (tags/v3.11.2:878ead1, Feb  7 2023, 16:38:35) [MSC v.1934 64 bit (AMD64)]


import sys
print(sys.version)